### Usage

This file is used to conduct combinatoric search for the amplifier and signal conditioner. First, we create all unique topologies (not considering doses) and then compute the objectives corresponding to different dose combinations using combinatorics.py and combinatorics_sigcond.py

Due to limitations on storage and computations, we split the topologies with corresponding doses and objectives into 54 shards for each problem. To compute the optimal solution(s), we find the optimal solution(s) in each shard, combine them, and then find the optimal solution(s) of the final set.

In [163]:
from GA import *
from itertools import combinations, permutations, product
import json
from amplifier_problem import Amplifier
from signal_conditioner_problem import SignalConditioner
from load_Z_mat_samples import Z_mat_list, Ref_list
from copy import deepcopy

In [2]:
def get_combinatorics(promo_node, num_part, has_inhibitor, tf_list, inhibitor_list):
    combo = []
    # all_edges = []
    if not has_inhibitor:
        #for i in range(1, max_part + 1):
        combo.extend(list(combinations(tf_list, num_part)))
    else:
        #for num_part in range(2, max_part + 1):
        for num_tf in range(1, num_part):
            num_in = num_part - num_tf
            list1 = combinations(tf_list, num_tf)
            list2 = combinations(inhibitor_list, num_in)
            combo_two = [(i[0] + i[1]) for i in list(product(list1, list2))]
            combo.extend(combo_two)

    circuits = []

    for i in range(len(combo)):
        part_list = combo[i]
        full_edge_list = get_full_connected(part_list, promo_node)
        dose_list = dict(zip(part_list, [0]*len(part_list)))
        circuits.append(Topo(full_edge_list, dose_list, promo_node))
        max_remove = len(full_edge_list) - 1

        for j in range(1, max_remove+1):
            combo_remove = list(combinations(full_edge_list, j))
            for k in combo_remove:
                edges = [e for e in full_edge_list if e not in k]
                # all_edges.append(edges)
                if check_valid(nx.DiGraph(edges), promo_node, part_list) == 1:
                    circuits.append(Topo(edges, dose_list, promo_node))
    return circuits #, all_edges

In [3]:
with open("parts.pkl", "rb") as fid:
    parts = pickle.load(fid)

In [4]:
tf_list = [k for k in parts.keys() if 'Z' in k]
inhibitor_list = [k for k in parts.keys() if 'I' in k]

In [40]:
promo_node = 'P1'
min_dose = 5
max_dose = 75
dose_step = 5
dose_range = [[k] for k in range(min_dose, max_dose+1, dose_step)]
dose_range_2 = [tuple([i, j]) for i in range(min_dose, max_dose+1, dose_step) for j in range(min_dose, max_dose+1, dose_step)]

In [11]:
num_part = 1
has_inhibitor = False
combo = get_combinatorics(promo_node, num_part, has_inhibitor, tf_list, inhibitor_list)
with open('./combo_search/main_combo_1.pkl', 'wb') as fid:
    pickle.dump(combo, fid)

num_part = 2
has_inhibitor = False
combo = get_combinatorics(promo_node, num_part, has_inhibitor, tf_list, inhibitor_list)
with open('./combo_search/main_combo_2.pkl', 'wb') as fid:
    pickle.dump(combo, fid)

num_part = 2
has_inhibitor = True
combo = get_combinatorics(promo_node, num_part, has_inhibitor, tf_list, inhibitor_list)
with open('./combo_search/main_combo_2_inhibitor.pkl', 'wb') as fid:
    pickle.dump(combo, fid)

In [153]:
if "Z_matrix" in settings:
    Z_mat = settings["Z_matrix"]
else:
    Z_mat = Z_mat_list[0]

if "reference" in settings:
    Ref_pop = settings["reference"]
else:
    Ref_pop = None

In [27]:
test_case = Amplifier

with open("./settings_amp_initial.json", "r") as fid:
    settings = json.load(fid)
    
problem = test_case(
        promo_node=settings["promo_node"],
        dose_specs=settings["dose_specs"],
        max_part=settings["max_part"],
        inhibitor=settings["inhibitor"],
        DsRed_inhibitor=settings["DsRed_inhibitor"],
        num_dict=settings["num_dict"],
        n_gen=settings["n_gen"],
        probability_crossover=settings["probability_crossover"],
        probability_mutation=settings["probability_mutation"],
        mutate_dose=settings["mutate_dose"],
        mutate_scheme_prob=settings["mutate_scheme_prob"],
        pop=settings["pop"],
        Z_mat=Z_mat,
        Ref_pop=Ref_pop,
        num_processes=settings["num_processes"],
        obj_labels=settings["obj_labels"],
        max_time=settings["max_time"]
)

In [155]:
with open('./combo_search/main_combo_1.pkl', 'rb') as fid:
    combo = pickle.load(fid)

In [98]:
obj = []
combo_expanded = []
for topo in combo:
    for dose in dose_range:
        for i, part in enumerate(topo.part_list):
            topo.dose[part] = dose[i]
        combo_expanded.append(deepcopy(topo))
        obj.append(problem.simulate_cell(topo))
obj = np.asarray(obj)

In [99]:
with open('./combo_search/amplifier/amplifier_topo_1.pkl', 'wb') as fid:
    pickle.dump(combo_expanded, fid)
with open('./combo_search/amplifier/amplifier_obj_1.pkl', 'wb') as fid:
    pickle.dump(obj, fid)

In [160]:
test_case = SignalConditioner

with open("./settings_sigcond.json", "r") as fid:
    settings = json.load(fid)
    
problem = test_case(
        promo_node=settings["promo_node"],
        dose_specs=settings["dose_specs"],
        max_part=settings["max_part"],
        inhibitor=settings["inhibitor"],
        DsRed_inhibitor=settings["DsRed_inhibitor"],
        num_dict=settings["num_dict"],
        n_gen=settings["n_gen"],
        probability_crossover=settings["probability_crossover"],
        probability_mutation=settings["probability_mutation"],
        mutate_dose=settings["mutate_dose"],
        mutate_scheme_prob=settings["mutate_scheme_prob"],
        pop=settings["pop"],
        Z_mat=Z_mat,
        Ref_pop=Ref_pop,
        num_processes=settings["num_processes"],
        obj_labels=settings["obj_labels"],
        max_time=settings["max_time"]
)

In [161]:
obj = []
combo_expanded = []
for topo in combo:
    for dose in dose_range:
        for i, part in enumerate(topo.part_list):
            topo.dose[part] = dose[i]
        combo_expanded.append(deepcopy(topo))
        obj.append(problem.simulate_cell(topo))
obj = np.asarray(obj)

In [162]:
with open('./combo_search/sigcond/amplifier_topo_1.pkl', 'wb') as fid:
    pickle.dump(combo_expanded, fid)
with open('./combo_search/sigcond/amplifier_obj_1.pkl', 'wb') as fid:
    pickle.dump(obj, fid)